# Неявное решение уравнения фильтрации однофазной жидкости в двухмерном пласте

Муравцев Александр. Вариант 16

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Задание значений входных параметров и перевод единиц измерения в СИ

In [2]:
mesh_x_count = 4  # количество ячеек по пространству в направлении оси Ox
mesh_y_count = 4  # количество ячеек по пространству в направлении оси Oy
mesh_t_count = 1000  # количество шагов по времени

In [3]:
dt = 10  # сут, временной шаг
dt *= 24 * 60 * 60  # с, временной шаг

m = 0.18  # пористость

mu = 4  # мПа*с, вязкость жидкости
mu *= 1e-3  # Па*с, вязкость жидкости

k = np.array([
    [30.0, 30.0, 30.0, 30.0],
    [30.0, 40.0, 40.0, 40.0],
    [30.0, 40.0, 40.0, 50.0],
    [30.0, 40.0, 50.0, 40.0]
])  # мД, проницаемость
k *= 1e-3 * (1e-6) ** 2  # м^2, проницаемость

dx = 100  # м, ширина ячейки по оси Ox
dy = 100  # м, ширина ячейки по оси Oy

h = 50  # м, толщина пласта

b_liq = 1  # м^3/м^3, объёмный коэффициент жидкости

compr = 2.2e-9  # Па^(-1), общая сжимаемость

p_initial = 20  # МПа, начальное пластовое давление
p_initial *= 1e6  # Па, начальное пластовое давление

q_inj = 40  # м^3/сут, расход нагнетательной скважины в поверхностных условиях
q_inj /= (24 * 60 * 60)  # м^3/с, расход нагнетательной скважины в поверхностных условиях

q_prod = 40  # м^3/сут, дебит добывающей скважины в поверхностных условиях
q_prod /= (24 * 60 * 60)  # м^3/с, дебит добывающей скважины в поверхностных условиях

In [4]:
# безразмерные множители, входящие в формулы неяного метода
gamma = (m * mu * compr / k) * (dx ** 2 / dt)
beta = (dx / dy) ** 2

In [5]:
p = []
p.append(np.full((mesh_x_count, mesh_y_count), p_initial))

In [6]:
system_matrix = np.zeros((mesh_x_count * mesh_y_count, mesh_x_count * mesh_y_count))
d = np.zeros(mesh_x_count * mesh_y_count)

for i in range(mesh_x_count):
    for j in range(mesh_x_count):
        system_matrix[i * mesh_x_count + j, i * mesh_x_count + j] = -2 - 2 * beta - gamma[i, j]
        d[i * mesh_x_count + j] = -gamma[i, j] * p[-1][i, j] 
system_matrix += np.diag([beta] * (mesh_x_count * mesh_y_count - 1), k=-1)
system_matrix += np.diag([beta] * (mesh_x_count * mesh_y_count - 1), k=1)
system_matrix += np.diag([1] * (mesh_y_count * mesh_x_count - mesh_x_count), k=mesh_x_count)
system_matrix += np.diag([1] * (mesh_y_count * mesh_x_count - mesh_x_count), k=-mesh_x_count)
system_matrix

array([[-4.61111111,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        , -4.61111111,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  1.        , -4.61111111,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        , -4.61111111,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.        ,  0

In [7]:
# ГУ в ячейке 2
system_matrix[1, 0] = 0
system_matrix[1, 1] = -1 - gamma[0, 1]
system_matrix[1, 5] = 1
system_matrix[1, 2] = 0

# ГУ в ячейке 3
system_matrix[2, 1] = 0
system_matrix[2, 2] = -1 - gamma[0, 2]
system_matrix[2, 6] = 1
system_matrix[2, 3] = 0

# ГУ в ячейке 14
system_matrix[13, 12] = 0
system_matrix[13, 9] = 1
system_matrix[13, 13] = -1 - gamma[3, 1]
system_matrix[13, 14] = 0

# ГУ в ячейке 15
system_matrix[14, 13] = 0
system_matrix[14, 10] = 1
system_matrix[14, 14] = -1 - gamma[3, 2]
system_matrix[14, 15] = 0

# ГУ в ячейке 1
system_matrix[0, 0] = -1 - beta - gamma[0, 0]
system_matrix[0, 4] = 1
system_matrix[0, 1] = beta

# ГУ в ячейке 16
system_matrix[15, 14] = beta
system_matrix[15, 11] = 1
system_matrix[15, 15] = -1 - beta - gamma[3, 3]

# ГУ в ячейке 5
system_matrix[4, 3] = 0
system_matrix[4, 0] = 0
system_matrix[4, 4] = -1 - gamma[1, 0] / beta
system_matrix[4, 8] = 0
system_matrix[4, 5] = 1

# ГУ в ячейке 9
system_matrix[8, 7] = 0
system_matrix[8, 4] = 0
system_matrix[8, 8] = -1 - gamma[2, 0] / beta
system_matrix[8, 12] = 0
system_matrix[8, 9] = 1

# ГУ в ячейке 8
system_matrix[7, 6] = 1
system_matrix[7, 3] = 0
system_matrix[7, 7] = -1 - gamma[1, 3] / beta
system_matrix[7, 11] = 0
system_matrix[7, 8] = 0

# ГУ в ячейке 12
system_matrix[11, 10] = 1
system_matrix[11, 7] = 0
system_matrix[11, 11] = -1 - gamma[2, 3] / beta
system_matrix[11, 15] = 0
system_matrix[11, 12] = 0

# ГУ в ячейке 4
system_matrix[3, 2] = beta
system_matrix[3, 3] = -1 - beta - gamma[0, 3]
system_matrix[3, 7] = 1
system_matrix[3, 4] = 0
d[3] = -gamma[0, 3] * p[-1][0, 3] + q_prod * mu / (dy * k[0, 3])

# ГУ в ячейке 13
system_matrix[12, 11] = 0
system_matrix[12, 8] = 1
system_matrix[12, 12] = -1 - beta - gamma[3, 0]
system_matrix[12, 13] = beta
d[12] = -gamma[3, 0] * p[-1][0, 3] - q_prod * mu / (dy * k[3, 0])


system_matrix

array([[-2.61111111,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -1.61111111,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        , -1.61111111,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        , -2.61111111,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0

In [8]:
p.append(np.linalg.solve(system_matrix, d).reshape(mesh_x_count, mesh_y_count))

In [9]:
p[-1]

array([[20000000.        , 20000000.        , 20000000.        ,
        19763593.38061466],
       [20000000.        , 20000000.        , 20000000.        ,
        20000000.        ],
       [20000000.        , 20000000.        , 20000000.        ,
        20000000.        ],
       [20236406.61938534, 19999999.99999999, 20000000.00000001,
        20000000.        ]])